In [1]:
from IPython.display import display
from IPython.core.display import HTML 
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)
 
import sqlite3
import unicodecsv as csv
import time
import datetime
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np
from __future__ import division

from ipywidgets import *

import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode()
pd.set_option('mode.chained_assignment',None)

Toggle code

In [2]:
import sys
sys.setrecursionlimit(20000)

In [64]:
import Tkinter as tk
import tkFileDialog

root = tk.Tk()
root.withdraw()
file_path = tkFileDialog.askopenfilename()

In [65]:
#con = sqlite3.connect('ASX.db')
#cur = con.cursor()

In [66]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days + 1)):
        yield start_date + datetime.timedelta(n)

In [67]:
def cagr(initial_amount, final_amount, years):
    '''calculate compound annual growth rate over a number of years, 
    given starting and ending equity'''
    return (final_amount / initial_amount) ** float(1 / years) - 1

In [68]:
HTML('''
<font face="calibri" color="black" size="5">
<h1><b>// TRADE SYSTEM PORTFOLIO TESTER</b></h1>

<p><font face="calibri" size="3"> This script takes a series of trades, and generates statistics about the performance of the system.</p>

<p><font face="calibri" size="3"> The input file should be a csv with columns Symbol,Entry_Date,Type,Entry_Price,stop_loss,initial_risk,Exit_Date,Exit_Price.</p>


''')

In [69]:
print "Input file: ", file_path

Input file:  C:/Users/Aaron/Documents/Trading/stop_loss_test.csv


In [70]:
def calc_unit_r_multiples(trades):
    '''takes in a DataFrame of trades and calculates the R multiple per unit
    DataFrame must contain columns '$_profit' and 'initial_risk' '''
    
    r_estimate = abs(trades[trades['$_profit'] <= 0]['$_profit'].mean())
    
    trades['unit_r_multiple'] = trades['$_profit'] / trades['initial_risk']
    
    r_multiples = []

    for index, row in trades.iterrows():

        if row['initial_risk'] == float(0):

            row['unit_r_multiple'] = row['$_profit'] / r_estimate

        r_multiples.append(row['unit_r_multiple'])
        
    trades['unit_r_multiple'] = r_multiples

In [71]:
def calc_r_multiples(trades):
    '''takes in a DataFrame of trades and calculates the R multiples
    DataFrame must contain columns 'total_$_profit' and 'initial_risk' '''
    
    r_estimate = abs(trades[trades['total_$_profit'] <= 0]['total_$_profit'].mean())
    
    trades['r_multiple'] = trades['total_$_profit'] / (trades['initial_risk'] * trades['position_size'])
    
    r_multiples = []

    for index, row in trades.iterrows():

        if row['initial_risk'] == float(0):

            row['r_multiple'] = row['total_$_profit'] / r_estimate

        r_multiples.append(row['r_multiple'])
        
    trades['r_multiple'] = r_multiples

In [72]:
def max_drawdown(r_multiples):
    '''calculate the max drawdown in terms of R produced by the system'''
    max_drawdown = 0
    in_drawdown = False
    current_drawdown = 0
    max_losses = 0
    current_losses = 0
    for r in r_multiples:
        
        if r < 0:
            in_drawdown = True
        else:
            in_drawdown = False
            
        if in_drawdown:
            current_drawdown += r
            current_losses += 1
        else:
            current_drawdown = 0
            current_losses = 0
            
        #print current_drawdown
        if current_losses > max_losses:
            max_losses = current_losses
            
        if current_drawdown < max_drawdown:
            max_drawdown = current_drawdown
        
    return max_drawdown, max_losses

In [73]:
def loss_win_ratio(r_multiples):
    losses = 0
    for r in r_multiples:
        if r < 0:
            losses += 1
            
    return float(losses / len(r_multiples))

In [74]:
def prob_of_streak(trades, min_losses, loss_prob, saved=None):
    '''refer to http://www.askamathematician.com/2010/07/q-whats-the-chance-of-getting-a-run-of-k-successes-in-n-bernoulli-trials-why-use-approximations-when-the-exact-answer-is-known/'''
    #for first call initialise a dict to save values
    if saved == None: saved = {}
        
    ID = (trades, min_losses, loss_prob)
    
    #check the dict for the ID value
    if ID in saved: return saved[ID]
    else:
        #handle the base case where we have no trades or losing streak exceeds number fo trades
        if min_losses > trades or trades <= 0:
            result = 0
            
        else:
            result = loss_prob ** min_losses
            for first_win in xrange(1, min_losses + 1):
                pr = prob_of_streak(trades - first_win, min_losses, loss_prob, saved)
                result += (loss_prob ** (first_win -1)) * (1 - loss_prob) * pr
                
                
        saved[ID] = result
        
        return result

In [75]:
def streaks(num_trades, loss_percentage, limit=50):
    '''returns an array of losing streaks and their probabilities.
    Default limit is 50 due to performance issues above this.'''
    streaks = pd.Series(
        data=[prob_of_streak(num_trades, i, loss_percentage) for i in range(1, min(num_trades,limit) + 1)],
        index = [i for i in range(1, min(num_trades,limit) + 1)])
   
    return streaks

In [76]:
def longest_streak(num_trades, loss_percentage, confidence):
    '''calculates the longest expected losing streak in a run of n trades, at a given confidence level,
    i.e. there is an x% chance you will get a streak of n losing trades'''
    
    losing_streaks = (streaks(num_trades, loss_percentage))
    try:
        streak = np.argmax((losing_streaks[losing_streaks <= confidence])) - 1
    except ValueError:
        streak = 0        
    return streak

In [77]:
def create_pos_df(positions, columns=None):
    '''takes in a dictionary of trade positions and creates a DataFrame with columns
    specified in the arguments'''
    
    poslist = []
    
    for k, v in positions.items():
        #print v
        poslist.append(v[0])
    
    positions_df = pd.DataFrame(poslist, columns=columns)
    
    return positions_df

In [78]:
#Monte Carlo function to simulate variability of returns over a given number of trades

def trade_monte_carlo(r_multiples,num_trades):
    ''' This function takes in an array of R Multiples and returns an array of n trades taken from the 
    input distribution'''
    
    # Define a trades array
    trades = np.random.choice(r_multiples, num_trades, replace=True)
        
    return trades



In [79]:
#Monte Carlo function to simulate n runs of x trades.

def system_monte_carlo(runs, r_multiples, num_trades, confidence):
    '''simulates multiple runs of x trades from an input R Mutiple distribution. Returns a DataFrame of expectancies,
    standard deviations of the R multilpes, total profit (in R), max drawdown (in R), loss/win ratio, 
    longest guaranteed losing streak at confidence% chance'''
    t0 = time.time()
    expectancies = []
    std_devs = []
    profits = []
    drawdowns = []
    ratios = []
    losing_streaks = []
    max_losses = []
    trials = []

    for run in xrange(runs):
        trades = trade_monte_carlo(r_multiples, num_trades)
        dd, ls = max_drawdown(trades)
        expectancies.append(trades.mean())
        std_devs.append(trades.std())
        profits.append(trades.sum())
        drawdowns.append(dd)
        ratios.append(loss_win_ratio(trades))
        losing_streaks.append(longest_streak(len(trades), loss_win_ratio(trades), confidence))
        max_losses.append(ls)
        trials.append(trades)
        
    results = pd.DataFrame()
    results['Expectancy'] = expectancies
    results['Std_dev'] = std_devs
    results['Profit'] = profits
    results['Drawdown'] = drawdowns
    results['loss_ratio'] = ratios
    results['losing_streak'] = losing_streaks
    results['max_losses'] = max_losses
    
    runtime = (time.time() - t0) / 60.0
    
    return results, trials, runtime

In [80]:
def portfolio_mc_run(trades, start_date, end_date, capital, portions, pyramid=True):
    '''simulates a single run of trades taken from an input trade database (DataFrame) 
    using an equal portions position sizing model. The simulation will take every entry signal on days when there is 
    enough available capital. For days where there are more entry signals than available positions, 
    the simulation randomly chooses trades up to the maximum number of available positions.
    
    The simulation uses a simple equal portions position sizing model, with the option to pyramid 
    closed trade profits only'''
    
    open_positions = {}
    closed_positions = {}
    closed_equity = capital
    cash = capital
    daily_balances = []
    
    if start_date < trades['Entry_Date'].min() or end_date > max(trades['Entry_Date'].max(), trades['Exit_Date'].max()):
        print "start or end date is outside the range of the trade database"
        raise Exception('bad date')
    
    for day in daterange(start_date, end_date):
        entry_signals = trades[trades['Entry_Date'] == day]
        exit_signals = trades[trades['Exit_Date'] == day]
        #print 'date: ', day
        #print 'cash balance: ', cash

        for symbol in exit_signals['Symbol']:

            if symbol in open_positions:

                closed_positions[symbol] = open_positions.pop(symbol, None)
                #closed_positions[symbol][0]['Exit_Price'] = exit_signals[exit_signals['Symbol'] == symbol]['Exit_Price']
                proceeds = closed_positions[symbol][0]['Exit_Price'] * closed_positions[symbol][0]['position_size']
                #print 'sale proceeds: ', proceeds
                cash += proceeds
                closed_equity += closed_positions[symbol][0]['$_profit'] * closed_positions[symbol][0]['position_size']

        #print 'closed trade equity: ', closed_equity
        if pyramid:
            portion_size = int(closed_equity / portions)
        else:
            portion_size = int(capital / portions)
        #available_positions = portions - len(open_positions)

        available_positions = int(cash / portion_size)

        #print "Available positions to buy: ", available_positions
        #print "Position size: ", portion_size

        #keep only those signals for symbols we don't already own
        entry_signals = entry_signals[~entry_signals['Symbol'].isin(open_positions.keys())]

        if available_positions >= len(entry_signals):

            for symbol in entry_signals['Symbol']:

                open_positions[symbol] = entry_signals[entry_signals['Symbol'] == symbol].to_dict(orient='records')
                open_positions[symbol][0]['position_size'] = int(portion_size / open_positions[symbol][0]['Entry_Price'])
                cash -= open_positions[symbol][0]['position_size'] * open_positions[symbol][0]['Entry_Price']

        else:
            '''handle the case where we have more entry signals than available cash to take the trades. 
            This is the key to MC analysis at a portfolio level.'''

            possible_trades = np.random.choice(entry_signals.index, available_positions, replace=False)
            #print available_positions, possible_trades
            for i in possible_trades:
                symbol = entry_signals.loc[i]['Symbol']
                open_positions[symbol] = entry_signals[entry_signals['Symbol'] == symbol].to_dict(orient='records')
                open_positions[symbol][0]['position_size'] = int(portion_size / open_positions[symbol][0]['Entry_Price'])
                cash -= open_positions[symbol][0]['position_size'] * open_positions[symbol][0]['Entry_Price']

        #print open_positions
        #print closed_positions
        daily_balances.append({'day' : day, 'cash' : cash, 'closed_equity' : closed_equity})
      
    daily_balances_df = pd.DataFrame(daily_balances, columns=['day', 'cash', 'closed_equity'])
    
    closed_positions_df = create_pos_df(closed_positions, columns=['Symbol', 'Entry_Date', 'Entry_Price', 'stop_loss'
                            , 'initial_risk', 'position_size', 'Exit_Date', 'Exit_Price', '$_profit', '%_profit'
                            , 'hold_time', 'r_multiple'])
    
    closed_positions_df['total_$_profit'] = closed_positions_df['$_profit'] * closed_positions_df['position_size']
    
    calc_r_multiples(closed_positions_df)
    
    open_positions_df = create_pos_df(open_positions, columns=['Symbol', 'Entry_Date', 'Entry_Price', 'stop_loss'
                            , 'initial_risk', 'position_size', 'Exit_Date', 'Exit_Price', '$_profit', '%_profit'
                            , 'hold_time', 'r_multiple'])
    
    return closed_positions_df, open_positions_df, daily_balances_df

In [81]:
#Monte Carlo function to simulate n runs of portfolio trades.

def portfolio_monte_carlo(runs, trade_db, start_date, end_date, capital, portions, pyramid=True, confidence=0.05):
    '''Simulates multiple runs of trades from an input trade database. The tester will randomly select trades signaled 
    on the same day to simulate real portfolio trading with limited capital. It uses the equal portions position 
    size model to determine the amount of capital to commit to a trade.
    
    Returns a DataFrame of expectancies,
    standard deviations of the R multiples, total profit (in R), max drawdown (in R), loss/win ratio, 
    longest guaranteed losing streak at confidence% chance'''
    t0 = time.time()
    expectancies = []
    std_devs = []
    profits = []
    drawdowns = []
    ratios = []
    losing_streaks = []
    max_losses = []
    equity_curves = []
    r_multiple_distributions = []
    trials = []

    for run in xrange(runs):
        trades, open_trades, bals = portfolio_mc_run(trade_db, start_date, end_date, capital, portions, pyramid=True)
        dd, ls = max_drawdown(trades['total_$_profit'])
        expectancies.append(trades['r_multiple'].mean())
        std_devs.append(trades['r_multiple'].std())
        r_multiple_distributions.append(trades['r_multiple'])
        profits.append(trades['total_$_profit'].sum())
        drawdowns.append(dd)
        ratios.append(loss_win_ratio(trades['total_$_profit']))
        losing_streaks.append(longest_streak(len(trades), loss_win_ratio(trades['total_$_profit']), confidence))
        max_losses.append(ls)
        equity_curves.append(bals)
        trials.append(trades)
        
    results = pd.DataFrame()
    results['Expectancy'] = expectancies
    results['Std_dev'] = std_devs
    results['Profit'] = profits
    results['Drawdown'] = drawdowns
    results['loss_ratio'] = ratios
    results['losing_streak'] = losing_streaks
    results['max_losses'] = max_losses
    
    runtime = (time.time() - t0) / 60.0
    
    return results, r_multiple_distributions, equity_curves, trials, runtime

In [82]:
#read the raw trades from a csv file
trades = pd.read_csv(file_path)
#trades.head()

In [83]:
try:
    trades['Entry_Date'] = pd.to_datetime(trades['Entry_Date'], format='%d/%m/%Y')
except:
    try:
        trades['Entry_Date'] = pd.to_datetime(trades['Entry_Date'], format='%Y-%m-%d')
    except:
        raise Exception
        
try:
    trades['Exit_Date'] = pd.to_datetime(trades['Exit_Date'], format='%d/%m/%Y')
except:
    try:
        trades['Exit_Date'] = pd.to_datetime(trades['Exit_Date'], format='%Y-%m-%d')
    except:
        trades['Exit_Date'] = pd.to_datetime(trades['Exit_Date'], format='%d/%m/%Y %H:%M')
        
#trades['Exit_Date'] = pd.to_datetime(trades['Exit_Date'], format='%d/%m/%Y')

In [84]:
print "Dates range from %s to %s" % (trades['Entry_Date'].min(), max(trades['Entry_Date'].max(), trades['Exit_Date'].max()))

Dates range from 2013-01-14 00:00:00 to 2016-12-28 00:00:00


In [85]:
#trades.head()

In [86]:
#calculate trade metrics - $/% profit/loss, hold time - and add to the DF
trades['$_profit'] = trades['Exit_Price'] - trades['Entry_Price']
trades['%_profit'] = trades['$_profit'] / trades['Entry_Price']
trades['hold_time'] = trades['Exit_Date'] - trades['Entry_Date']
#trades.head()

In [87]:
trades.fillna(value=0, inplace=True)

In [88]:
r_estimate = abs(trades[trades['$_profit'] <= 0]['$_profit'].mean())
r_estimate

0.5339045601794149

In [89]:
calc_unit_r_multiples(trades)

In [90]:
#trades.head()

In [91]:
r_multiples = trades['unit_r_multiple']

In [92]:
r_dist = go.Histogram(
    x = r_multiples.values,
    name = 'R Multiple Distribution',
    autobinx=False,
    xbins=dict(
        start=int(r_multiples.values.min()),
        end=int(r_multiples.values.max()),
        size=0.5
    )
)

r_dist_layout = go.Layout(
    
    title='Distribution of Raw Input R Multiples',
    bargap=0.2
)

r_dist_plot = go.Figure(data=[r_dist], layout=r_dist_layout)
iplot(r_dist_plot)

In [93]:
in_dd, in_ls = max_drawdown(r_multiples)

In [94]:
HTML('''
<table  style="width:80%%">
  <tr>
    <td>Total trades in input sample: </td>
    <td><b>%s</b></td> 
    <td>Win rate of input sample: </td>
    <td><b>%s</b></td>
  </tr>
  <tr>
    <td>Expectancy (mean R Multiple) of input sample: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of R in input sample: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Max Drawdown (in R) in input sample: </td>
    <td><b>%s</b></td>
    <td>Longest losing streak in input sample: </td>
    <td><b>%s</b></td> 
  </tr>
</table>


'''% ('{:,}'.format(len(r_multiples)), '{:.2%}'.format(1 - loss_win_ratio(r_multiples)), 
      '{:.4}'.format(np.mean(r_multiples)), '{:.4}'.format(np.std(r_multiples)),
      '{:.4}'.format(in_dd), '{:,}'.format(in_ls))  )


In [95]:
capital = 100000
portions = 10
portion_size = capital / portions

In [96]:
mc_runs = 100
mc_confidence = 0.05
start_date = datetime.datetime(2013,1,18) 
end_date = datetime.datetime(2016,12,1)

In [97]:
years = (end_date - start_date).days / 365

In [98]:
mc_result, r_multiple_distributions, equity_curves, mc_trials, mc_runtime = portfolio_monte_carlo(mc_runs, trades
                                                , start_date, end_date, capital, portions, pyramid=True
                                                , confidence=mc_confidence)

In [99]:
trades_taken = []
for r in r_multiple_distributions:
    trades_taken.append(len(r))

In [100]:
cagrs = []
for profit in mc_result['Profit']:
    cagrs.append(cagr(capital, (capital + profit), years))

In [101]:
HTML('''
<font face="calibri" color="black" size="5">
<h1><b>// MONTE CARLO TEST</b></h1>

<p><font face="calibri" size="3">Monte Carlo test run for <b>%s</b> simulations on input file %s.</p>
<p>Test period is from <b>%s</b> to <b>%s</b></p>
<p>The system is profitable in <b>%s</b> of runs</p>
<p>&nbsp</p>
<p>Based on the average win rate and average number of trades per run, 
there is a <b>95%%</b> chance the system will have at least <b>%s</b> losses in a row 
and a <b>%s</b> chance of <b>%s</b> losses in a row </p>
<p>&nbsp</p>
<p>Test completed in <b>%s</b> minutes</p>
<p>&nbsp</p>
<table  style="width:80%%">
  <tr>
    <td>Initial Capital: </td>
    <td><b>%s</b></td>
  </tr>
  <tr>
    <td>Max Simultaneous Positions: </td>
    <td><b>%s</b></td>
  </tr>
</table>
<p>&nbsp</p>
<table  style="width:80%%">
  <tr>
    <td>Average trades per run: </td>
    <td><b>%s</b></td> 
    <td>Average Win Rate: </td>
    <td><b>%s</b></td>
  </tr>
    <tr>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
  </tr>
  <tr>
    <td>Average Expectancy: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of Expectancy: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Minimum Expectancy: </td>
    <td><b>%s</b></td>
    <td>Maximum Expectancy: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
  </tr>  

    <tr>
    <td>Average Total Profit: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of Profit: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Minimum Total Profit: </td>
    <td><b>%s</b></td>
    <td>Maximum Total Profit: </td>
    <td><b>%s</b></td> 
  </tr>
    <tr>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
  </tr>
  <tr>
    <td>Average CAGR: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of CAGR: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Minimum CAGR: </td>
    <td><b>%s</b></td>
    <td>Maximum CAGR: </td>
    <td><b>%s</b></td> 
  </tr>
    <tr>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
  </tr>
  <tr>
    <td>Average Max Drawdown: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of Max Drawdown: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Minimum Max Drawdown: </td>
    <td><b>%s</b></td>
    <td>Maximum Max Drawdown: </td>
    <td><b>%s</b></td> 
  </tr>
    <tr>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
    <td>&nbsp</td>
  </tr>
  <tr>
    <td>Average Longest losing streak: </td>
    <td><b>%s</b></td>
    <td>Standard deviation of Longest losing streak: </td>
    <td><b>%s</b></td> 
  </tr>
  <tr>
    <td>Minimum Longest losing streak: </td>
    <td><b>%s</b></td>
    <td>Maximum Longest losing streak: </td>
    <td><b>%s</b></td> 
  </tr>    
  </tr>
</table>

''' % ('{:,}'.format(mc_runs), '{}'.format(file_path)
        ,'{:%d-%m-%Y}'.format(start_date), '{:%d-%m-%Y}'.format(end_date)
        , '{:.2%}'.format(len(mc_result['Profit'][mc_result['Profit'] > 0]) / len(mc_result))
        , '{:,}'.format(longest_streak(int(np.mean(trades_taken)), np.mean(mc_result['loss_ratio']), 0.95))
        , '{:.2%}'.format(mc_confidence)
        , '{:,}'.format(longest_streak(int(np.mean(trades_taken)), np.mean(mc_result['loss_ratio']), 0.05))
        , '{:.4}'.format(mc_runtime)
        , '{:,}'.format(capital), '{:,}'.format(portions) 
        , '{:,}'.format(np.mean(trades_taken)), '{:.2%}'.format(1 - np.mean(mc_result['loss_ratio']))
        , '{:.2}'.format(np.mean(mc_result['Expectancy'])), '{:.2}'.format(np.std(mc_result['Expectancy']))
        , '{:.2}'.format(min(mc_result['Expectancy'])), '{:.2}'.format(max(mc_result['Expectancy']))
        , '{:,}'.format(int(np.mean(mc_result['Profit']))), '{:,}'.format(int(np.std(mc_result['Profit'])))
       , '{:,}'.format(int(min(mc_result['Profit']))), '{:,}'.format(int(max(mc_result['Profit'])))
       , '{:.2%}'.format(np.mean(cagrs)), '{:.2%}'.format(np.std(cagrs))
       , '{:.2%}'.format(min(cagrs)), '{:.2%}'.format(max(cagrs))
       , '{:,}'.format(int(np.mean(mc_result['Drawdown']))), '{:,}'.format(int(np.std(mc_result['Drawdown'])))
       , '{:,}'.format(int(max(mc_result['Drawdown']))), '{:,}'.format(int(min(mc_result['Drawdown'])))
       , '{:,}'.format(np.mean(mc_result['max_losses'])), '{:.2}'.format(np.std(mc_result['max_losses']))
       , '{:,}'.format(min(mc_result['max_losses'])), '{:,}'.format(max(mc_result['max_losses']))
))

In [102]:
num_trades = go.Histogram(
    x = trades_taken,
    name = 'Trades Taken distribution'
)

num_trades_layout = go.Layout(
    
    title='Distribution of Number of Trades Taken',
    bargap=0.2,
)

num_trades_plot = go.Figure(data=[num_trades], layout=num_trades_layout)
iplot(num_trades_plot)

In [103]:
HTML('''
<font face="calibri" color="black" size="5">
<h1><b>// Expectancy</b></h1>

<p><font face="calibri" size="3">Expectancy describes the expected return of the system over the long term.
It is the amount you expect to earn per dollar risked.</p>
<p>&nbsp</p>
<p>The formula for expectancy is Average Profit per trade / Average Loss, 
where Average Profit per trade = Net Profit / Total Trades</p>
<p>&nbsp</p>
<p>The expectancy is also equal to the mean R Multiple of all the trades</p>

''' )

In [104]:
expectancies = go.Histogram(
    x = mc_result['Expectancy'],
    name = 'Expectancy distribution'
)

layout = go.Layout(
    
    title='Distribution of Expectancies (Mean R Multiple)',
    bargap=0.2,
)

expectancy_plot = go.Figure(data=[expectancies], layout=layout)
iplot(expectancy_plot)

In [105]:
std_devs = go.Histogram(
    x = mc_result['Std_dev'],
    name = 'Standard Deviation Distribution'
)

std_devs_layout = go.Layout(
    
    title='Distribution of Standard Deviation of R-Multiples',
    bargap=0.2,
)

std_devs_plot = go.Figure(data=[std_devs], layout=std_devs_layout)
iplot(std_devs_plot)

In [106]:
profits = go.Histogram(
    x = mc_result['Profit'],
    name = 'Total Profit Distribution'
)

profits_layout = go.Layout(
    
    title='Distribution of Total Profits ($)',
    bargap=0.2,
)

profits_plot = go.Figure(data=[profits], layout=profits_layout)
iplot(profits_plot)

In [107]:
cagrs_trace = go.Histogram(
    x = cagrs,
    name = 'CAGR %'
)

cagrs_layout = go.Layout(
    
    title='Compound Annual Growth Rate %',
    bargap=0.2,
)

cagrs_plot = go.Figure(data=[cagrs_trace], layout=cagrs_layout)
iplot(cagrs_plot)

In [108]:
drawdowns = go.Histogram(
    x = mc_result['Drawdown'],
    name = 'Max Drawdown Distribution'
)

drawdowns_layout = go.Layout(
    
    title='Distribution of Max Drawdowns ($)',
    bargap=0.2,
)

drawdowns_plot = go.Figure(data=[drawdowns], layout=drawdowns_layout)
iplot(drawdowns_plot)

In [109]:
loss_ratios = go.Histogram(
    x = 1 - mc_result['loss_ratio'],
    name = 'Win Rate Distribution'
)

loss_ratios_layout = go.Layout(
    
    title='Distribution of Win Rate',
    bargap=0.2,
)

loss_ratios_plot = go.Figure(data=[loss_ratios], layout=loss_ratios_layout)
iplot(loss_ratios_plot)

In [110]:
losing_streaks_proj = go.Histogram(
    x = mc_result['losing_streak'],
    name = 'Theoretical Max Losing Streak'
)

losing_streaks_act = go.Histogram(
    x = mc_result['max_losses'],
    name = 'Actual Max Losing Streak'
)

streaks_data = [losing_streaks_proj, losing_streaks_act]

losing_streaks_layout = go.Layout(
    
    title='Distribution of Longest Losing Streaks (Theoretical and Actual)',
    bargap=0.2,
)

losing_streaks_plot = go.Figure(data=streaks_data, layout=losing_streaks_layout)
iplot(losing_streaks_plot)

In [111]:
#trialsdf = pd.DataFrame(data=mc_trials)

In [112]:
traces = []

for i in xrange(min(len(equity_curves), 20)):
    
    traces.append(go.Scatter(
        x = equity_curves[i]['day'],
        y = equity_curves[i].closed_equity,
        name = 'trial' + str(i)
            ,        mode = 'lines'
            #,       line = dict(color = colors[i])
    ))



equity_fig = go.Figure(data=traces
                        #, layout=layout
                       )

equity_fig['layout'].update(title = 'Equity curves per Trial (first 20 trials)',
                           xaxis = {'title' : 'Trades'},
                           yaxis = {'title' : 'Closed Trade Equity ($)'})

iplot(equity_fig)

In [113]:
cash_traces = []

for i in xrange(min(len(equity_curves), 20)):
    
    cash_traces.append(go.Scatter(
        x = equity_curves[i]['day'],
        y = equity_curves[i].cash,
        name = 'trial' + str(i)
            ,        mode = 'lines'
            #,       line = dict(color = colors[i])
    ))



cash_fig = go.Figure(data=cash_traces
                        #, layout=layout
                       )

cash_fig['layout'].update(title = 'Cash Balance per Trial (first 20 trials)',
                           xaxis = {'title' : 'Trades'},
                           yaxis = {'title' : 'Cash Balance ($)'})

iplot(cash_fig)

In [53]:
#closed_pos.to_csv(path_or_buf=join('C:\Users\Aaron\Documents\Trading', 'trade_db.csv'), index=False)